The Goal is to summarize a dialogue without prompt engineering with the pretrained LLM FLAN-T5 from Hugging Face.  

In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, GenerationConfig

/opt/anaconda3/envs/GenAI_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
huggingface_dataset_name = "knkarthick/dialogsum"
dataset = load_dataset(huggingface_dataset_name)

The dataset has "id", "dialogue", "summary", and "topic" columns, all in string format. 

In [3]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
})


In [4]:
print(dataset["train"][0])

{'id': 'train_0', 'dialogue': "#Person1#: Hi, Mr. Smith. I'm Doctor Hawkins. Why are you here today?\n#Person2#: I found it would be a good idea to get a check-up.\n#Person1#: Yes, well, you haven't had one for 5 years. You should have one every year.\n#Person2#: I know. I figure as long as there is nothing wrong, why go see the doctor?\n#Person1#: Well, the best way to avoid serious illnesses is to find out about them early. So try to come at least once a year for your own good.\n#Person2#: Ok.\n#Person1#: Let me see here. Your eyes and ears look fine. Take a deep breath, please. Do you smoke, Mr. Smith?\n#Person2#: Yes.\n#Person1#: Smoking is the leading cause of lung cancer and heart disease, you know. You really should quit.\n#Person2#: I've tried hundreds of times, but I just can't seem to kick the habit.\n#Person1#: Well, we have classes and some medications that might help. I'll give you more information before you leave.\n#Person2#: Ok, thanks doctor.", 'summary': "Mr. Smith's 

In [5]:
dataset["train"].to_pandas().head(1)

,id,dialogue,summary,topic
0,train_0,"#Person1#: Hi, Mr. Smith. I'm Doctor Hawkins. ...","Mr. Smith's getting a check-up, and Doctor Haw...",get a check-up


In [6]:
example_indices = [40, 200]
dash_line = '-'.join('' for x in range(100))

for i, index in enumerate(example_indices):
    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print('INPUT DIALOGUE:')
    print(dataset['test'][index]['dialogue'])
    print(dash_line)
    print('BASELINE HUMAN SUMMARY:')
    print(dataset['test'][index]['summary'])
    print(dash_line)
    print()

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT DIALOGUE:
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------

---------------------------------------------------------------------------------------------------
Exa

Load FLAN-T5 model. 

In [7]:
model_name = "google/flan-t5-base"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/opt/anaconda3/envs/GenAI_env/lib/python3.10/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/opt/anaconda3/envs/GenAI_env/lib/python3.10/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

/opt/anaconda3/envs/GenAI_env/lib/python3.10/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [12]:
sentence = "What time is it, Tom?"
sentence_encoded = tokenizer(sentence, return_tensors='pt')
# return_tensors='pt'   return a PyTorch tensor output. 
print(sentence_encoded)

{'input_ids': tensor([[ 363,   97,   19,   34,    6, 3059,   58,    1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}


In [14]:
print('ENCODED SENTENCE: ')
print(sentence_encoded["input_ids"][0])

ENCODED SENTENCE: 
tensor([ 363,   97,   19,   34,    6, 3059,   58,    1])


In [15]:
sentence_decoded = tokenizer.decode(sentence_encoded["input_ids"][0], 
                                    skip_special_tokens = True)
print("DECODED SENTENCE: ")
print(sentence_decoded)

DECODED SENTENCE: 
What time is it, Tom?


Explore how well the LLM summarizes a dialogue without any prompt engineering. 

In [19]:
for i, index in enumerate(example_indices):
    dialogue = dataset["test"][index]['dialogue']
    summary = dataset['test'][index]['summary']

    inputs = tokenizer(dialogue, return_tensors='pt')
    outputs = tokenizer.decode(
        model.generate(inputs["input_ids"],
                       max_new_tokens = 50,
                       )[0],
                       skip_special_tokens=True
    )
    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print(f'INPUT Prompt: \n{dialogue}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n {summary}')
    print(dash_line)
    print(f'MODEL GENERATION - WITHOUT PROMPT ENGINEERING: \n {outputs}\n')

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT Prompt: 
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
 #Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
MODEL GENERATION - WITHOUT PROMPT ENGINEERING: 
 Person1: It's ten to nine.

---------------------------

### Summerize Dialogue with an Instruction Prompt

Zero-shot inference 

In [22]:
for i, index in enumerate(example_indices):
    dialogue = dataset["test"][index]['dialogue']
    summary = dataset['test'][index]['summary']

    prompt = f"""
Summarize the following conversation.

{dialogue}

Summary:
"""
    inputs = tokenizer(prompt, return_tensors='pt')
    outputs = tokenizer.decode(
        model.generate(inputs["input_ids"],
                       max_new_tokens = 50,
                       )[0],
                       skip_special_tokens=True
    )
    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print(f'INPUT PROMPT: \n{prompt}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n {summary}')
    print(dash_line)
    print(f'MODEL GENERATION - ZERO SHOT: \n {outputs}\n')

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT: 

Summarize the following conversation.

#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

Summary:

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
 #Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
MODEL GENERATION - ZERO SHOT: 
 The train is about to

In [23]:
for i, index in enumerate(example_indices):
    dialogue = dataset["test"][index]['dialogue']
    summary = dataset['test'][index]['summary']

    prompt = f"""
Dialogue:

{dialogue}

What was going on?
"""
    inputs = tokenizer(prompt, return_tensors='pt')
    outputs = tokenizer.decode(
        model.generate(inputs["input_ids"],
                       max_new_tokens = 50,
                       )[0],
                       skip_special_tokens=True
    )
    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print(f'INPUT PROMPT: \n{prompt}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n {summary}')
    print(dash_line)
    print(f'MODEL GENERATION - ZERO SHOT: \n {outputs}\n')

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT: 

Dialogue:

#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

What was going on?

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
 #Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
MODEL GENERATION - ZERO SHOT: 
 Tom is late for the train.

-----------

Still Not Great!!!!

In [26]:
def make_prompt(example_indices_full, example_index_to_summarize):
    prompt = ''
    for index in example_indices_full:
        dialogue = dataset["test"][index]['dialogue']
        summary = dataset['test'][index]['summary']

    prompt = f"""
Dialogue:

{dialogue}

What was going on?
{summary}


"""
    dialogue = dataset["test"][example_index_to_summarize]['dialogue']
    prompt += f"""
Dialogue:

{dialogue}

What was going on?
"""
    return prompt

Construct the prompt ofr one-shot inference:

In [27]:
example_indices_full = [40]
example_index_to_summarize = 200

one_shot_prompt = make_prompt(example_indices_full, example_index_to_summarize)

print(one_shot_prompt)


Dialogue:

#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

What was going on?
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.



Dialogue:

#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to begin with. And you also ne

In [28]:
summary = dataset['test'][example_index_to_summarize]['summary']

inputs = tokenizer(one_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens = 50,
    )[0],
    skip_special_tokens =True
)
print(dash_line)
print(f'BASELINE HUMAN SUMMARY: \n {summary}\n')
print(dash_line)
print(f'MODEL GENERATION - ONE SHOT: \n {output}')

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY: 
 #Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.

---------------------------------------------------------------------------------------------------
MODEL GENERATION - ONE SHOT: 
 #Person1 wants to upgrade his system. #Person2 wants to add a painting program to his software. #Person1 wants to add a CD-ROM drive.


Few shot inference:

In [29]:
example_indices_full = [40, 80, 120]
example_index_to_summarize = 200

few_shot_prompt = make_prompt(example_indices_full, example_index_to_summarize)

print(few_shot_prompt)


Dialogue:

#Person1#: Hello, I bought the pendant in your shop, just before. 
#Person2#: Yes. Thank you very much. 
#Person1#: Now I come back to the hotel and try to show it to my friend, the pendant is broken, I'm afraid. 
#Person2#: Oh, is it? 
#Person1#: Would you change it to a new one? 
#Person2#: Yes, certainly. You have the receipt? 
#Person1#: Yes, I do. 
#Person2#: Then would you kindly come to our shop with the receipt by 10 o'clock? We will replace it. 
#Person1#: Thank you so much. 

What was going on?
#Person1# wants to change the broken pendant in #Person2#'s shop.



Dialogue:

#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#

In [30]:
summary = dataset['test'][example_index_to_summarize]['summary']

inputs = tokenizer(few_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens = 50,
    )[0],
    skip_special_tokens =True
)
print(dash_line)
print(f'BASELINE HUMAN SUMMARY: \n {summary}\n')
print(dash_line)
print(f'MODEL GENERATION - FEW SHOT: \n {output}')

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY: 
 #Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.

---------------------------------------------------------------------------------------------------
MODEL GENERATION - FEW SHOT: 
 #Person1 wants to upgrade his computer. #Person2 wants to add a painting program to his software. #Person1 wants to add a CD-ROM drive.


In this case, the few shot did not provide much of an improvement over one shot inference. 

### Generative Configuraton Parameters for Inference

In [31]:
generation_config = GenerationConfig(max_new_tokens = 50)
# generation_config = GenerationConfig(max_new_tokens = 10)
# generation_config = GenerationConfig(max_new_tokens = 50, do_sample=True, temperature=0.1)
# generation_config = GenerationConfig(max_new_tokens = 50, do_sample=True, temperature=0.5)
# generation_config = GenerationConfig(max_new_tokens = 50, do_sample=True, temperature=1.0)
inputs = tokenizer(few_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        generation_config=generation_config,
    )[0],
    skip_special_tokens =True
)
print(dash_line)
print(f'BASELINE HUMAN SUMMARY: \n {summary}\n')
print(dash_line)
print(f'MODEL GENERATION - FEW SHOT: \n {output}')

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY: 
 #Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.

---------------------------------------------------------------------------------------------------
MODEL GENERATION - FEW SHOT: 
 #Person1 wants to upgrade his computer. #Person2 wants to add a painting program to his software. #Person1 wants to add a CD-ROM drive.


Choosing max_new_tokens = 10 will make the output text too short, so the dialogue summary will be cut. 

Putting do_sample=True and changing the temperature value you get more flexibility in the output. 

temperature = 2.0 is the highest, gives you very wild responses. 